# "backtesting crypto with exit signal"
> "How to fetch and backtest crypto data using fastquant"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Mikee Jazmines
- categories: [crypto, backtest]

<a href="https://colab.research.google.com/github/enzoampil/fastquant/blob/master/examples/2020-05-20-backtest_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment to install in colab
# !pip3 install fastquant --update
# or pip install git+https://www.github.com/enzoampil/fastquant.git@history

## fetch data from binance

### If a timestamp is given, it will return upto that timestamp

In [2]:
from fastquant import get_crypto_data

In [3]:
crypto = get_crypto_data("ADA/USDT", 
                         "2022-01-01 00:00:00", 
                         "2022-03-03 00:00:00",
                         time_resolution='1h'
                        )

In [4]:
crypto.tail()

,open,high,low,close,volume
dt,,,,,
2022-03-02 20:00:00,0.935,0.938,0.931,0.935,2939304.5
2022-03-02 21:00:00,0.936,0.947,0.933,0.946,2943418.6
2022-03-02 22:00:00,0.946,0.952,0.940,0.943,2918147.9
2022-03-02 23:00:00,0.943,0.946,0.936,0.939,2451045.1
2022-03-03 00:00:00,0.938,0.946,0.931,0.940,4678771.1


## EMA Crossover Example

In [5]:
from fastquant import backtest

In [6]:
# Import modules
import backtrader as bt

# Import from package
from fastquant.strategies.base import BaseStrategy


class EMACStrategy(BaseStrategy):
    """
    Exponential moving average crossover strategy

    Parameters
    ----------
    fast_period : int
        The period used for the fast exponential moving average line (should be smaller than `slow_upper`)
    slow_period : int
        The period used for the slow exponential moving average line (should be larger than `fast_upper`)

    """

    params = (
        ("fast_period", 10),  # period for the fast moving average
        ("slow_period", 30),
    )

    def __init__(self):
        # Initialize global variables
        super().__init__()
        # Strategy level variables
        self.fast_period = self.params.fast_period
        self.slow_period = self.params.slow_period
        self.short_max = 1

        if self.strategy_logging:
            print("===Strategy level arguments===")
            print("fast_period :", self.fast_period)
            print("slow_period :", self.slow_period)
            print("allow_short :", self.allow_short)
        ema_fast = bt.ind.EMA(period=self.fast_period)  # fast moving average
        ema_slow = bt.ind.EMA(period=self.slow_period)  # slow moving average
        self.crossover = bt.ind.CrossOver(
            ema_fast, ema_slow
        )  # crossover signal

    def buy_signal(self):
        return self.crossover > 0

    def sell_signal(self):
        return self.crossover < 0

### Long only

In [7]:
results, history = backtest(EMACStrategy, 
                               crypto, 
                            fast_period = 20,
                            slow_period = 50,
                               plot=False,
                               verbose=False,
                               return_history=True,
                            # allow_short=True,
                              )

In [8]:
orders = history['orders']
orders

,strat_id,strat_name,dt,type,price,size,order_value,portfolio_value,commission,pnl
0,0,fast_period20_slow_period50,2022-01-05 11:00:00,buy,1.345,74275,99899.875,98737.325,0.0,0.000
1,0,fast_period20_slow_period50,2022-01-05 13:00:00,sell,1.329,-74275,99899.875,98811.600,0.0,-1188.400
2,0,fast_period20_slow_period50,2022-01-11 20:00:00,buy,1.187,83161,98712.107,98894.761,0.0,0.000
3,0,fast_period20_slow_period50,2022-01-19 06:00:00,sell,1.409,-83161,98712.107,117273.342,0.0,18461.742
4,0,fast_period20_slow_period50,2022-01-26 13:00:00,buy,1.096,106894,117155.824,118662.964,0.0,0.000
5,0,fast_period20_slow_period50,2022-01-27 05:00:00,sell,1.024,-106894,117155.824,109576.974,0.0,-7696.368
6,0,fast_period20_slow_period50,2022-01-29 07:00:00,buy,1.057,103564,109467.148,111233.998,0.0,0.000
7,0,fast_period20_slow_period50,2022-01-30 20:00:00,sell,1.031,-103564,109467.148,106884.310,0.0,-2692.664
8,0,fast_period20_slow_period50,2022-02-01 04:00:00,buy,1.056,101115,106777.440,106783.195,0.0,0.000
9,0,fast_period20_slow_period50,2022-02-02 22:00:00,sell,1.032,-101115,106777.440,104457.550,0.0,-2426.760


The final value in `results` can be calculated from the `commission` and `pnl` (profit & loss) of all the closed (bought and sold) transactions in history:

In [9]:
r = results.squeeze()
r.final_value

101885.39199999996

In [10]:
profit = (r.final_value - 100000)/100000*100
print("Strategy")
print(f"Profit of asset: {profit}")

Strategy
Profit of asset: 1.8853919999999635


In [11]:
print("Buy and hold")
print(f"Start price of asset: {crypto.iloc[0, 3]}")
print(f"End price of asset: {crypto.iloc[-1, 3]}")
print(f"Profit of asset: {(crypto.iloc[-1, 3] - crypto.iloc[0, 3])/crypto.iloc[0, 3]*100}")

Buy and hold
Start price of asset: 1.334
End price of asset: 0.94
Profit of asset: -29.5352323838081


### Long and Short

In [12]:
results, history = backtest(EMACStrategy, 
                               crypto, 
                            fast_period = 20,
                            slow_period = 50,
                               plot=False,
                               verbose=False,
                               return_history=True,
                             allow_short=True,
                              )

In [13]:
orders = history['orders']
orders

,strat_id,strat_name,dt,type,price,size,order_value,portfolio_value,commission,pnl
0,0,fast_period20_slow_period50,2022-01-03 22:00:00,sell,1.324,-75585,-100074.540,100075.585,0.0,0.000
1,0,fast_period20_slow_period50,2022-01-05 11:00:00,buy,1.345,148605,-1862.640,97171.375,0.0,-1587.285
2,0,fast_period20_slow_period50,2022-01-05 13:00:00,sell,1.329,-146411,675.261,97537.959,0.0,-1168.320
3,0,fast_period20_slow_period50,2022-01-11 20:00:00,buy,1.187,163931,9934.341,107756.457,0.0,10421.522
4,0,fast_period20_slow_period50,2022-01-19 06:00:00,sell,1.409,-182325,-21854.085,129326.142,0.0,20099.880
5,0,fast_period20_slow_period50,2022-01-26 13:00:00,buy,1.096,234337,26911.927,158347.678,0.0,28728.705
6,0,fast_period20_slow_period50,2022-01-27 05:00:00,sell,1.024,-285216,10149.056,146088.094,0.0,-10263.744
7,0,fast_period20_slow_period50,2022-01-29 07:00:00,buy,1.057,276278,-4857.938,143660.670,0.0,-4707.912
8,0,fast_period20_slow_period50,2022-01-30 20:00:00,sell,1.031,-266609,4112.153,137117.917,0.0,-3473.964
9,0,fast_period20_slow_period50,2022-02-01 04:00:00,buy,1.056,260314,-2668.981,134596.688,0.0,-3324.875


In [14]:
periodic = history['periodic']
periodic.loc[periodic['dt']>='2022-01-30 19:00:00'].head(20)

,strat_id,strat_name,dt,portfolio_value,cash,size,return
673,0,fast_period20_slow_period50,2022-01-30 19:00:00,138048.882,292.848,133614,-0.013369
674,0,fast_period20_slow_period50,2022-01-30 20:00:00,137117.917,275166.727,-132995,-0.006744
675,0,fast_period20_slow_period50,2022-01-30 21:00:00,137649.897,275166.727,-132995,0.003880
676,0,fast_period20_slow_period50,2022-01-30 22:00:00,137383.907,275166.727,-132995,-0.001932
677,0,fast_period20_slow_period50,2022-01-30 23:00:00,137516.902,275166.727,-132995,0.000968
678,0,fast_period20_slow_period50,2022-01-31 00:00:00,140043.807,275166.727,-132995,0.018375
679,0,fast_period20_slow_period50,2022-01-31 01:00:00,140043.807,275166.727,-132995,0.000000
680,0,fast_period20_slow_period50,2022-01-31 02:00:00,138979.847,275166.727,-132995,-0.007597
681,0,fast_period20_slow_period50,2022-01-31 03:00:00,139245.837,275166.727,-132995,0.001914
682,0,fast_period20_slow_period50,2022-01-31 04:00:00,140442.792,275166.727,-132995,0.008596


In [15]:
periodic.loc[periodic['dt']>='2022-02-01 03:00:00'].head(20)

,strat_id,strat_name,dt,portfolio_value,cash,size,return
705,0,fast_period20_slow_period50,2022-02-01 03:00:00,134724.007,275166.727,-132995,-0.010742
706,0,fast_period20_slow_period50,2022-02-01 04:00:00,134596.688,275.143,127319,-0.000945
707,0,fast_period20_slow_period50,2022-02-01 05:00:00,134342.050,275.143,127319,-0.001892
708,0,fast_period20_slow_period50,2022-02-01 06:00:00,133832.774,275.143,127319,-0.003791
709,0,fast_period20_slow_period50,2022-02-01 07:00:00,134724.007,275.143,127319,0.006659
710,0,fast_period20_slow_period50,2022-02-01 08:00:00,135487.921,275.143,127319,0.005670
711,0,fast_period20_slow_period50,2022-02-01 09:00:00,133960.093,275.143,127319,-0.011276
712,0,fast_period20_slow_period50,2022-02-01 10:00:00,133960.093,275.143,127319,0.000000
713,0,fast_period20_slow_period50,2022-02-01 11:00:00,134596.688,275.143,127319,0.004752
714,0,fast_period20_slow_period50,2022-02-01 12:00:00,135615.240,275.143,127319,0.007567


The final value in `results` can be calculated from the `commission` and `pnl` (profit & loss) of all the closed (bought and sold) transactions in history:

In [16]:
r = results.squeeze()
r.final_value

133244.36700000003

In [17]:
profit2 = (r.final_value - 100000)/100000*100
print("Strategy")
print(f"Profit of asset: {profit2}")

Strategy
Profit of asset: 33.24436700000003


In [18]:
print("Buy and hold")
print(f"Start price of asset: {crypto.iloc[0, 3]}")
print(f"End price of asset: {crypto.iloc[-1, 3]}")
print(f"Profit of asset: {(crypto.iloc[-1, 3] - crypto.iloc[0, 3])/crypto.iloc[0, 3]*100}")

Buy and hold
Start price of asset: 1.334
End price of asset: 0.94
Profit of asset: -29.5352323838081


## Final Results:

In [19]:
print(f"Buy and hold profit %: {round((crypto.iloc[-1, 3] - crypto.iloc[0, 3])/crypto.iloc[0, 3]*100, 2)}%")
print(f"Long only profit %: {round(profit, 2)}%")
print(f"Long and Short profit %: {round(profit2, 2)}%")

Buy and hold profit %: -29.54%
Long only profit %: 1.89%
Long and Short profit %: 33.24%
